<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

# 2. Armado del ambiente

## 2.1 Armado del ambiente en CPU

No se requiere ejecuciones previas para la ejecución del programa

##2.2 Armado del ambiente en GPU

Se requiere la instalación del modulo de Cuda para Python

In [18]:
!pip install pycuda


#3. Desarrollo

##3.1 Desarrollo en CPU

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  5#@param {type: "number"}
orden_taylor = 10#@param {type: "number"}
x0 =  3.5#@param {type: "number"}
x = 4#@param {type: "number"}
# --------------------------------------------

import numpy
import time
import math

def factorial(x):
  if x == 0 or x == 1:
    return 1
  factorial = 1
  for i in range (1, x+1):
    factorial = factorial * i
  return factorial

tiempoTotal = time.time() * 1000

coeficientes = numpy.random.randint(10, size = orden_polinomio +1)
#coeficientes = numpy.array([3, 2, 3, 4])

longitud_poli = len(coeficientes)

derivadas = numpy.arange(longitud_poli)

print("Coeficientes:\n", coeficientes)

polin_taylor = numpy.zeros(orden_taylor + 1 , dtype=float)
longitud_taylor = len(polin_taylor)
print("Orden polinomio", longitud_poli - 1 )
print("Longitud_Taylor:", longitud_taylor)
print("*****************************************************************")

tiempoBucle = time.time() *1000
#----------------------ALGORITMO----------------------------
for idx_t in range(0, longitud_taylor):
  valor = 0
  aux = 0
  
  if idx_t == 0:
    polin_taylor[idx_t] += coeficientes[idx_t]
    idx_p = idx_t + 1
    for idx in range(idx_p, longitud_poli):
      polin_taylor[idx_t] += coeficientes[idx] * math.pow(x0, derivadas[idx])
      print("idt:", idx_t, ":idx:", idx, "|", "aux:", polin_taylor[idx_t])
  else:
    if idx_t < longitud_poli - 1 :
      aux += coeficientes[idx_t]
      idx_p = idx_t + 1
      resta = math.pow(x - x0, idx_t)
      f = factorial(idx_t)
      for idx in range(idx_p, longitud_poli):
        termino = coeficientes[idx] * derivadas[idx]
        coeficientes[idx] = termino
        derivadas[idx] = derivadas[idx] - 1
        factor = math.pow(x0, derivadas[idx])
        aux += termino * factor        
        print("idt:", idx_t, ":idx:", idx,"|",
              "aux:", aux , "termino:", termino)
      polin_taylor[idx_t] = (aux * resta) / f        
    else:
      if idx_t >= longitud_poli: 
        print("idx_t:", idx_t)
        valor =  0
        polin_taylor[idx_t] = valor
        print("Valor:", valor)
      else:
        print("idx_t:", idx_t)
        valor =  coeficientes[idx_t]
        resta = math.pow(x - x0, idx_t)
        f = factorial(idx_t)
        polin_taylor[idx_t] = (valor * resta)/ f
        print("Valor:", valor)
  print("-----------------------------------------------------------------")
#----------------------FINALIZACION----------------------------

tiempoBucle = (time.time() *1000) - tiempoBucle
print("Termino el algoritmo")
print(polin_taylor)

tiempoTotal = (time.time() *1000) - tiempoTotal
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

Coeficientes:
 [4 2 4 5 1 4]
Orden polinomio 5
Longitud_Taylor: 11
*****************************************************************
idt: 0 :idx: 1 | aux: 11.0
idt: 0 :idx: 2 | aux: 60.0
idt: 0 :idx: 3 | aux: 274.375
idt: 0 :idx: 4 | aux: 424.4375
idt: 0 :idx: 5 | aux: 2525.3125
-----------------------------------------------------------------
idt: 1 :idx: 2 | aux: 30.0 termino: 8
idt: 1 :idx: 3 | aux: 213.75 termino: 15
idt: 1 :idx: 4 | aux: 385.25 termino: 4
idt: 1 :idx: 5 | aux: 3386.5 termino: 20
-----------------------------------------------------------------
idt: 2 :idx: 3 | aux: 113.0 termino: 30
idt: 2 :idx: 4 | aux: 260.0 termino: 12
idt: 2 :idx: 5 | aux: 3690.0 termino: 80
-----------------------------------------------------------------
idt: 3 :idx: 4 | aux: 114.0 termino: 24
idt: 3 :idx: 5 | aux: 3054.0 termino: 240
-----------------------------------------------------------------
idt: 4 :idx: 5 | aux: 1704.0 termino: 480
---------------------------------------------------

##3.2 Desarrollo en GPU

In [19]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio = 50000#@param {type: "number"}
orden_taylor = 1#@param {type: "number"}
x0 = 1#@param {type: "number"}
x = 1#@param {type: "number"}
# --------------------------------------------

import numpy
import time

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempoTotal = (time.time() * 1000)

# --------------------------------------------
# CPU - Defino la memoria de los vectores en cpu.
#coeficientes_cpu = numpy.random.randn(orden_polinomio + 1)
coeficientes_cpu = numpy.array([3, 2, 3, 4])
coeficientes_cpu = coeficientes_cpu.astype(numpy.float32())
long_pol = len(coeficientes_cpu)

taylor_cpu = numpy.zeros(orden_taylor +1, dtype=float)
taylor_cpu = taylor_cpu.astype(numpy.float32())
long_tay = len(coeficientes_cpu)

derivadas_cpu = numpy.arange(long_pol)
derivadas_cpu = derivadas_cpu.astype(numpy.int32)

resultado_cpu = numpy.empty_like(taylor_cpu)
print("C:", coeficientes_cpu)
print("T:", taylor_cpu)
print("D:", derivadas_cpu)
print("R:", resultado_cpu)

# CPU - reservo la memoria GPU.
coeficientes_gpu = cuda.mem_alloc( coeficientes_cpu.nbytes )
taylor_gpu = cuda.mem_alloc( taylor_cpu.nbytes )
derivadas_gpu = cuda.mem_alloc( derivadas_cpu.nbytes )
# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( coeficientes_gpu, coeficientes_cpu )
cuda.memcpy_htod( taylor_gpu, taylor_cpu )
cuda.memcpy_htod( derivadas_gpu, derivadas_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_taylor( int n, float alfa, float *X, float *Y )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    Y[idx]  = alfa*X[idx] + Y[idx];
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_taylor")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N),numpy.float32(alfa), x_gpu, y_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( r_cpu, y_gpu )

tiempoTotal = time.time() *1000 - tiempoTotal

print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempoTotal, "[ms]" )
print("Tiempo GPU: ", tiempo_gpu, "[ms]" )

C: [3. 2. 3. 4.]
T: [0. 0.]
D: [0 1 2 3]
R: [0.    1.875]
Thread x:  256 , Bloque x: 196
Cantidad de elementos:  50000
Thread x:  256 , Bloque x: 196
Tiempo CPU:  5.732177734375 [ms]
Tiempo GPU:  0:00:00.000518 [ms]


#4. Tabla de pasos

#5. Conclusiones